In [ ]:
import pyspark.sql.functions as fn
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "clientID",
"fs.azure.account.oauth2.client.secret": "secretValue",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{tenantID}/oauth2/token"}

dbutils.fs.mount(
source = "abfss://tokyo-olympic-data@tokyoolympicdatastorage1.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolympic3",
extra_configs = configs)

True

In [ ]:
%fs
ls "/mnt/tokyoolympic3"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic3/raw-data/,raw-data/,0,1695394870000
dbfs:/mnt/tokyoolympic3/transformed-data/,transformed-data/,0,1695394880000


In [ ]:
spark

In [ ]:
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/mnt/tokyoolympic3/raw-data/athletes.csv')
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/mnt/tokyoolympic3/raw-data/coaches.csv')
entries_gender = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/mnt/tokyoolympic3/raw-data/entries_gender.csv')
medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/mnt/tokyoolympic3/raw-data/medals.csv')
teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/mnt/tokyoolympic3/raw-data/teams.csv')

In [ ]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank_by_Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
# entries_gender = entries_gender.withColumn("Female", fn.col("Female").cast(IntegerType())) \
#     .withColumn("Male", fn.col("Male").cast(IntegerType())) \
#     .withColumn("Total", fn.col("Total").cast(IntegerType()))

In [ ]:
entries_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank_by_Total: integer (nullable = true)



In [ ]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [ ]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country", "Gold").show(truncate=False)

+--------------------------+----+
|Team_Country              |Gold|
+--------------------------+----+
|United States of America  |39  |
|People's Republic of China|38  |
|Japan                     |27  |
|Great Britain             |22  |
|ROC                       |20  |
|Australia                 |17  |
|Netherlands               |10  |
|France                    |10  |
|Germany                   |10  |
|Italy                     |10  |
|Canada                    |7   |
|Brazil                    |7   |
|New Zealand               |7   |
|Cuba                      |7   |
|Hungary                   |6   |
|Republic of Korea         |6   |
|Poland                    |4   |
|Czech Republic            |4   |
|Kenya                     |4   |
|Norway                    |4   |
+--------------------------+----+
only showing top 20 rows



In [ ]:
# Calculate the average number of entries by gender for each discipline
avg_entries_by_gender = entries_gender.withColumn("Avg_Female", fn.round(entries_gender['Female'] / entries_gender['Total'], 3)) \
    .withColumn("Avg_Male", fn.round(entries_gender['Male'] / entries_gender['Total'], 3))

avg_entries_by_gender.show()

+--------------------+------+----+-----+----------+--------+
|          Discipline|Female|Male|Total|Avg_Female|Avg_Male|
+--------------------+------+----+-----+----------+--------+
|      3x3 Basketball|    32|  32|   64|       0.5|     0.5|
|             Archery|    64|  64|  128|       0.5|     0.5|
| Artistic Gymnastics|    98|  98|  196|       0.5|     0.5|
|   Artistic Swimming|   105|   0|  105|       1.0|     0.0|
|           Athletics|   969|1072| 2041|     0.475|   0.525|
|           Badminton|    86|  87|  173|     0.497|   0.503|
|   Baseball/Softball|    90| 144|  234|     0.385|   0.615|
|          Basketball|   144| 144|  288|       0.5|     0.5|
|    Beach Volleyball|    48|  48|   96|       0.5|     0.5|
|              Boxing|   102| 187|  289|     0.353|   0.647|
|        Canoe Slalom|    41|  41|   82|       0.5|     0.5|
|        Canoe Sprint|   123| 126|  249|     0.494|   0.506|
|Cycling BMX Frees...|    10|   9|   19|     0.526|   0.474|
|  Cycling BMX Racing|  

In [ ]:
# Send transformed data to the bucket
athletes.repartition(1).write.mode("overwrite").option("header", "true").parquet("/mnt/tokyoolympic3/transformed-data/athletes.parquet")
coaches.repartition(1).write.mode("overwrite").option("header", "true").parquet("/mnt/tokyoolympic3/transformed-data/coaches.parquet")
entries_gender.repartition(1).write.mode("overwrite").option("header", "true").parquet("/mnt/tokyoolympic3/transformed-data/entries_gender.parquet")
medals.repartition(1).write.mode("overwrite").option("header", "true").parquet("/mnt/tokyoolympic3/transformed-data/medals.parquet")
teams.repartition(1).write.mode("overwrite").option("header", "true").parquet("/mnt/tokyoolympic3/transformed-data/teams.parquet")